## Install PySpark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 261 kB in 1s (212 kB/s)
Reading package lis

In [2]:
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [4]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.3.1-bin-hadoop3'

In [5]:
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark import SparkFiles


spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

spark

## Read Data

In [6]:
spark=SparkSession.builder.getOrCreate()
## ensure to use the link of the raw file of the csv
url = "https://raw.githubusercontent.com/VAKGL/SparkML-and-SparkSQL/main/SparkML/HeartStroke.csv" # Make sure the url is the raw version of the file on GitHub


In [7]:
spark.sparkContext.addFile(url)

In [8]:
# Reading the downloaded content and turning it into a pandas dataframe

df = spark.read.csv(SparkFiles.get("HeartStroke.csv"),inferSchema=True, header= True)

In [9]:
df.show(5)

+------+----+--------+------------+------+-------------+---------------+-----+
|gender| age|diabetes|hypertension|stroke|heart_disease|smoking_history|  BMI|
+------+----+--------+------------+------+-------------+---------------+-----+
|Female|80.0|       0|           0|    No|          Yes|          never|25.19|
|Female|54.0|       0|           0|    No|           No|           null| null|
|  Male|28.0|       0|           0|    No|           No|          never| null|
|Female|36.0|       0|           0|    No|           No|        current|23.45|
|  Male|76.0|       0|           1|    No|          Yes|        current|20.14|
+------+----+--------+------------+------+-------------+---------------+-----+
only showing top 5 rows



In [10]:
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- stroke: string (nullable = true)
 |-- heart_disease: string (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- BMI: double (nullable = true)



In [12]:
stroke_df = df.toPandas()
stroke_df.head()

,gender,age,diabetes,hypertension,stroke,heart_disease,smoking_history,BMI
0,Female,80.0,0,0,No,Yes,never,25.19
1,Female,54.0,0,0,No,No,None,NaN
2,Male,28.0,0,0,No,No,never,NaN
3,Female,36.0,0,0,No,No,current,23.45
4,Male,76.0,0,1,No,Yes,current,20.14


In [13]:
df.describe().toPandas()

,summary,gender,age,diabetes,hypertension,stroke,heart_disease,smoking_history,BMI
0,count,100000,100000,100000,100000,100000,100000,64184,74556
1,mean,None,41.88585600000013,0.085,0.07485,None,None,None,27.321028891034764
2,stddev,None,22.51683987161704,0.27888308976661896,0.2631504702289171,None,None,None,7.686295651045002
3,min,Female,0.08,0,0,No,No,current,10.01
4,max,Other,80.0,1,1,Yes,Yes,not current,95.69


## Check missing values
(For this exercise we will drop all the na values)

In [14]:
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+------+---+--------+------------+------+-------------+---------------+-----+
|gender|age|diabetes|hypertension|stroke|heart_disease|smoking_history|  BMI|
+------+---+--------+------------+------+-------------+---------------+-----+
|     0|  0|       0|           0|     0|            0|          35816|25444|
+------+---+--------+------------+------+-------------+---------------+-----+



The columns 'smoking history' and 'BMI' has missing values. Let's drop them

In [15]:
# Use df_name.na.drop() to drop all the null values from the dataframe
df = df.na.drop()

In [16]:
# Check if the null value still exist
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+------+---+--------+------------+------+-------------+---------------+---+
|gender|age|diabetes|hypertension|stroke|heart_disease|smoking_history|BMI|
+------+---+--------+------------+------+-------------+---------------+---+
|     0|  0|       0|           0|     0|            0|              0|  0|
+------+---+--------+------------+------+-------------+---------------+---+



In [17]:
df.show(5, False)

+------+----+--------+------------+------+-------------+---------------+-----+
|gender|age |diabetes|hypertension|stroke|heart_disease|smoking_history|BMI  |
+------+----+--------+------------+------+-------------+---------------+-----+
|Female|80.0|0       |0           |No    |Yes          |never          |25.19|
|Female|36.0|0       |0           |No    |No           |current        |23.45|
|Male  |76.0|0       |1           |No    |Yes          |current        |20.14|
|Female|44.0|1       |0           |No    |No           |never          |19.31|
|Male  |42.0|0       |0           |No    |No           |never          |33.64|
+------+----+--------+------------+------+-------------+---------------+-----+
only showing top 5 rows



## Check and confirm the data type of each column

In [18]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- stroke: string (nullable = true)
 |-- heart_disease: string (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- BMI: double (nullable = true)



- Let us convert the columns "diabetes", "hypertension" data type into type double

In [19]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
intcols = ["diabetes", "hypertension"]

for col_name in intcols:
    strokeDF = df.withColumn(col_name, col(col_name).cast(DoubleType()))

# We will also rename the column name 'stroke' as 'label' for 

strokeDF.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- hypertension: double (nullable = true)
 |-- stroke: string (nullable = true)
 |-- heart_disease: string (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- BMI: double (nullable = true)



## Transformations

### Binarizer

- Let's use divide the BMi into two groups: Obese and healthy. 

- 1 represents 'obese' and 0 represents 'healthy' (If your BMI is 30.0 or higher, it falls within the obese range)

- We will use the Binarizer transformer to create a new variable 'Body Type' (1- obese and 0- healthy) by binarizing the 'BMI' variable by setting the obesity threshold value 30.0. 

- Binarization is used for thresholding numerical feature to binary feature (0 or 1)

In [20]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(inputCol="BMI", outputCol="BodyType", threshold=30.0)
binarizedDF = binarizer.transform(strokeDF)
binarizedDF.select('BMI', 'BodyType').show(5,False)

+-----+--------+
|BMI  |BodyType|
+-----+--------+
|25.19|0.0     |
|23.45|0.0     |
|20.14|0.0     |
|19.31|0.0     |
|33.64|1.0     |
+-----+--------+
only showing top 5 rows



In [21]:
binarizedDF.show(5)

+------+----+--------+------------+------+-------------+---------------+-----+--------+
|gender| age|diabetes|hypertension|stroke|heart_disease|smoking_history|  BMI|BodyType|
+------+----+--------+------------+------+-------------+---------------+-----+--------+
|Female|80.0|       0|         0.0|    No|          Yes|          never|25.19|     0.0|
|Female|36.0|       0|         0.0|    No|           No|        current|23.45|     0.0|
|  Male|76.0|       0|         1.0|    No|          Yes|        current|20.14|     0.0|
|Female|44.0|       1|         0.0|    No|           No|          never|19.31|     0.0|
|  Male|42.0|       0|         0.0|    No|           No|          never|33.64|     1.0|
+------+----+--------+------------+------+-------------+---------------+-----+--------+
only showing top 5 rows



From the above result we can see that the value of the target feature label is now converted to binary values

### Bucketizer

- We now group the patients based on their age group. 

- Here, we will use the Bucketizer transformer. 

- Bucketizer is used for creating group of values of a continuous feature

In [22]:
from pyspark.ml.feature import Bucketizer
# lets define the age age group splits
splits = [0, 25.0, 50.0, 75.0, 100.0]
bucketizer = Bucketizer(inputCol="age", outputCol="ageGroup", splits=splits)
bucketizedDF = bucketizer.transform(binarizedDF)
bucketizedDF.select('age', 'ageGroup').show(10,False)

+----+--------+
|age |ageGroup|
+----+--------+
|80.0|3.0     |
|36.0|1.0     |
|76.0|3.0     |
|44.0|1.0     |
|42.0|1.0     |
|54.0|2.0     |
|78.0|3.0     |
|67.0|2.0     |
|15.0|0.0     |
|42.0|1.0     |
+----+--------+
only showing top 10 rows



In [23]:
bucketizedDF.columns

['gender',
 'age',
 'diabetes',
 'hypertension',
 'stroke',
 'heart_disease',
 'smoking_history',
 'BMI',
 'BodyType',
 'ageGroup']

In [24]:
bucketizedDF.select('age', 'ageGroup').show(10,False)

+----+--------+
|age |ageGroup|
+----+--------+
|80.0|3.0     |
|36.0|1.0     |
|76.0|3.0     |
|44.0|1.0     |
|42.0|1.0     |
|54.0|2.0     |
|78.0|3.0     |
|67.0|2.0     |
|15.0|0.0     |
|42.0|1.0     |
+----+--------+
only showing top 10 rows



### StringIndexer

- There are three categorical variables in our dataset viz., 'gender', 'heart disease' and 'smoking history'. 

- These variables cannot be directly passed to our ML algorithms. 

- We will converet them into indexes and to do that we will use StringIndexer transformer. 

- StringIndexer converts a string column to an index column. The most frequent label gets index 0

In [25]:
from pyspark.ml.feature import StringIndexer
indexers = StringIndexer(inputCols= ['stroke','gender', 'heart_disease', 'smoking_history'], 
                         outputCols=['label', 'gender_indexed', 'heart_disease_indexed', 'smoking_history_indexed'])
strindexedDF = indexers.fit(bucketizedDF).transform(bucketizedDF)
strindexedDF.select('stroke', 'label', 'gender', 'gender_indexed', 'heart_disease', 'heart_disease_indexed', 
                    'smoking_history', 'smoking_history_indexed').show(5, False)

+------+-----+------+--------------+-------------+---------------------+---------------+-----------------------+
|stroke|label|gender|gender_indexed|heart_disease|heart_disease_indexed|smoking_history|smoking_history_indexed|
+------+-----+------+--------------+-------------+---------------------+---------------+-----------------------+
|No    |0.0  |Female|0.0           |Yes          |1.0                  |never          |0.0                    |
|No    |0.0  |Female|0.0           |No           |0.0                  |current        |2.0                    |
|No    |0.0  |Male  |1.0           |Yes          |1.0                  |current        |2.0                    |
|No    |0.0  |Female|0.0           |No           |0.0                  |never          |0.0                    |
|No    |0.0  |Male  |1.0           |No           |0.0                  |never          |0.0                    |
+------+-----+------+--------------+-------------+---------------------+---------------+--------

- From the above output you can observe that the categorical columns has got converted to their respective indices columns. 

- The most frequent label gets index 0 and so on ordered by label frequencies

### OneHotEncoderEstimator

In [26]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols= ["gender_indexed", 'heart_disease_indexed', 'smoking_history_indexed'], 
                         outputCols=["genderVec", 'heart_diseaseVec', 'smoking_historyVec'])
encodedDF = encoder.fit(strindexedDF).transform(strindexedDF)
encodedDF.select('gender_indexed', 'genderVec', 'heart_disease_indexed', 'heart_diseaseVec', 
                    'smoking_history_indexed', 'smoking_historyVec',).show(5, False)

+--------------+-------------+---------------------+----------------+-----------------------+------------------+
|gender_indexed|genderVec    |heart_disease_indexed|heart_diseaseVec|smoking_history_indexed|smoking_historyVec|
+--------------+-------------+---------------------+----------------+-----------------------+------------------+
|0.0           |(2,[0],[1.0])|1.0                  |(1,[],[])       |0.0                    |(4,[0],[1.0])     |
|0.0           |(2,[0],[1.0])|0.0                  |(1,[0],[1.0])   |2.0                    |(4,[2],[1.0])     |
|1.0           |(2,[1],[1.0])|1.0                  |(1,[],[])       |2.0                    |(4,[2],[1.0])     |
|0.0           |(2,[0],[1.0])|0.0                  |(1,[0],[1.0])   |0.0                    |(4,[0],[1.0])     |
|1.0           |(2,[1],[1.0])|0.0                  |(1,[0],[1.0])   |0.0                    |(4,[0],[1.0])     |
+--------------+-------------+---------------------+----------------+-----------------------+---

In [27]:
encodedDF.show(5)

+------+----+--------+------------+------+-------------+---------------+-----+--------+--------+-----+--------------+---------------------+-----------------------+-------------+----------------+------------------+
|gender| age|diabetes|hypertension|stroke|heart_disease|smoking_history|  BMI|BodyType|ageGroup|label|gender_indexed|heart_disease_indexed|smoking_history_indexed|    genderVec|heart_diseaseVec|smoking_historyVec|
+------+----+--------+------------+------+-------------+---------------+-----+--------+--------+-----+--------------+---------------------+-----------------------+-------------+----------------+------------------+
|Female|80.0|       0|         0.0|    No|          Yes|          never|25.19|     0.0|     3.0|  0.0|           0.0|                  1.0|                    0.0|(2,[0],[1.0])|       (1,[],[])|     (4,[0],[1.0])|
|Female|36.0|       0|         0.0|    No|           No|        current|23.45|     0.0|     1.0|  0.0|           0.0|                  0.0|     

### VectorAssembler
- MLlib expects all features to be contained within a single column. 
- VectorAssembler combines multiple columns and gives single column as output

In [28]:
# Import VectorAssembler from pyspark.ml.feature package
from pyspark.ml.feature import VectorAssembler
# Create a list of all the variables that you want to create feature vectors
# These features are then further used for training model
features_col = ["age", "diabetes", "hypertension", "BMI", "BodyType", "ageGroup", 
                "genderVec","heart_diseaseVec","smoking_historyVec"]
# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= features_col, outputCol= "features")
assembledDF = assembler.transform(encodedDF)
assembledDF.select("features").show(5, False)

+-----------------------------------------------------+
|features                                             |
+-----------------------------------------------------+
|(13,[0,3,5,6,9],[80.0,25.19,3.0,1.0,1.0])            |
|(13,[0,3,5,6,8,11],[36.0,23.45,1.0,1.0,1.0,1.0])     |
|(13,[0,2,3,5,7,11],[76.0,1.0,20.14,3.0,1.0,1.0])     |
|(13,[0,1,3,5,6,8,9],[44.0,1.0,19.31,1.0,1.0,1.0,1.0])|
|(13,[0,3,4,5,7,8,9],[42.0,33.64,1.0,1.0,1.0,1.0,1.0])|
+-----------------------------------------------------+
only showing top 5 rows



Now that all the features are vectorized. Let us convert the target variable 'stroke' (integer) in to a label column

All the features are now in one single feature vector. If you notice, the feature column contains sparse vector. In order to perform scaling on the data, we must convert the sparse vector to dense vector

In [29]:
assembledDF.columns

['gender',
 'age',
 'diabetes',
 'hypertension',
 'stroke',
 'heart_disease',
 'smoking_history',
 'BMI',
 'BodyType',
 'ageGroup',
 'label',
 'gender_indexed',
 'heart_disease_indexed',
 'smoking_history_indexed',
 'genderVec',
 'heart_diseaseVec',
 'smoking_historyVec',
 'features']

### VectorIndexer
VectorIndexer automatically identifies the categorical features from the feature vector (output from VectorAssembler). It then indexes categorical features inside of a Vector
It is the vectorized version of StringIndexer

In [30]:
# Import VectorIndexer from pyspark.ml.feature package
from pyspark.ml.feature import VectorIndexer
# Create a list of all the raw features
# VectorIndexer will automatically identify the categorical columns and index them
featurecol = ['age', 'diabetes','hypertension', 'BMI','BodyType','ageGroup', 
              "gender_indexed", 'heart_disease_indexed', 'smoking_history_indexed']

# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= featurecol, outputCol= "features")
assembledDF = assembler.transform(strindexedDF)

# Create the VectorIndexer object. It only take feature column
vecindexer = VectorIndexer(inputCol= "features", outputCol= "indexed_features")
# Fit the vectorindexer object on the output of the vectorassembler data and transform
vecindexedDF = vecindexer.fit(assembledDF).transform(assembledDF)
vecindexedDF.select("features", "indexed_features").show(5, False)

+----------------------------------------+----------------------------------------+
|features                                |indexed_features                        |
+----------------------------------------+----------------------------------------+
|(9,[0,3,5,7],[80.0,25.19,3.0,1.0])      |(9,[0,3,5,7],[80.0,25.19,3.0,1.0])      |
|(9,[0,3,5,8],[36.0,23.45,1.0,2.0])      |(9,[0,3,5,8],[36.0,23.45,1.0,2.0])      |
|[76.0,0.0,1.0,20.14,0.0,3.0,1.0,1.0,2.0]|[76.0,0.0,1.0,20.14,0.0,3.0,1.0,1.0,2.0]|
|(9,[0,1,3,5],[44.0,1.0,19.31,1.0])      |(9,[0,1,3,5],[44.0,1.0,19.31,1.0])      |
|[42.0,0.0,0.0,33.64,1.0,1.0,1.0,0.0,0.0]|[42.0,0.0,0.0,33.64,1.0,1.0,1.0,0.0,0.0]|
+----------------------------------------+----------------------------------------+
only showing top 5 rows



VectorIndexer let us skip the one hot encoding stage for encoding the categorical features. As discussed earlier, we should not use one hot encoding on categorical variables for algorithms like decision tree and tree ensembles. VectorIndexer are chosen over OneHotEncoderEstimator in such scenario which allows these algorithms to treat categorical features appropriately

### StandardScaler
StandardScaler scales each value in the feature vector such that the mean is 0 and the standard deviation is 1
<br>It takes parameters:
<br>withStd: True by default. Scales the data to unit standard deviation
<br>withMean: False by default. Centers the data with mean before scaling

**If you notice the output of vectorassembler data is a sparse vector. This need to be converted into a dense vector befor applying scaling**

In [31]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a udf that converts sparse vector into dense vector
# You cannot create your own custom function and run that against the data directly. 
# In Spark, You have to register the function first using udf function
sparseToDense = F.udf(lambda v : Vectors.dense(v), VectorUDT())

# We then call the function here passing the column name on which the function has to be applied
densefeatureDF = assembledDF.withColumn('features_array', sparseToDense('features'))

densefeatureDF.select("features", "features_array").show(5, False)

+----------------------------------------+----------------------------------------+
|features                                |features_array                          |
+----------------------------------------+----------------------------------------+
|(9,[0,3,5,7],[80.0,25.19,3.0,1.0])      |[80.0,0.0,0.0,25.19,0.0,3.0,0.0,1.0,0.0]|
|(9,[0,3,5,8],[36.0,23.45,1.0,2.0])      |[36.0,0.0,0.0,23.45,0.0,1.0,0.0,0.0,2.0]|
|[76.0,0.0,1.0,20.14,0.0,3.0,1.0,1.0,2.0]|[76.0,0.0,1.0,20.14,0.0,3.0,1.0,1.0,2.0]|
|(9,[0,1,3,5],[44.0,1.0,19.31,1.0])      |[44.0,1.0,0.0,19.31,0.0,1.0,0.0,0.0,0.0]|
|[42.0,0.0,0.0,33.64,1.0,1.0,1.0,0.0,0.0]|[42.0,0.0,0.0,33.64,1.0,1.0,1.0,0.0,0.0]|
+----------------------------------------+----------------------------------------+
only showing top 5 rows



In [32]:
densefeatureDF.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- hypertension: double (nullable = true)
 |-- stroke: string (nullable = true)
 |-- heart_disease: string (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- BMI: double (nullable = true)
 |-- BodyType: double (nullable = true)
 |-- ageGroup: double (nullable = true)
 |-- label: double (nullable = false)
 |-- gender_indexed: double (nullable = false)
 |-- heart_disease_indexed: double (nullable = false)
 |-- smoking_history_indexed: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- features_array: vector (nullable = true)



In [33]:
# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
stdscaler = StandardScaler(inputCol= "features_array", outputCol= "scaledfeatures")

# Fit the StandardScaler object on the output of the dense vector data and transform
stdscaledDF = stdscaler.fit(densefeatureDF).transform(densefeatureDF)
stdscaledDF.select("scaledfeatures" ).show(5, False)

+-----------------------------------------------------------------------------------------------------------------------------------------+
|scaledfeatures                                                                                                                           |
+-----------------------------------------------------------------------------------------------------------------------------------------+
|[4.087032486459775,0.0,0.0,3.4969597048301693,0.0,3.465141331509375,0.0,4.588062443054963,0.0]                                           |
|[1.839164618906899,0.0,0.0,3.255407109101527,0.0,1.155047110503125,0.0,0.0,1.5657422910924597]                                           |
|[3.882680862136787,0.0,3.266884631584384,2.795901883893593,0.0,3.465141331509375,2.0514943144150433,4.588062443054963,1.5657422910924597]|
|[2.2478678675528765,3.122394188242338,0.0,2.680678519264413,0.0,1.155047110503125,0.0,0.0,0.0]                                           |
|[2.145692055391382,

## Spark ML Algorithms
We will now train the ML models with the data that we have worked upon so far. We will build classification model since, given the data, we need to determine if a person will get a stroke or not

### Train-Test Split
We split the output of  data into training and test sets (30% held out for testing)
Note: This train-test split of for logistic regression

In [34]:
# We spilt the data into 70-30 set
# Training Set - 70% obesevations
# Testing Set - 30% observations
trainDF, testDF =  assembledDF.randomSplit([0.7,0.3], seed = 2020)

# print the count of observations in each set
print("Observations in training set = ", trainDF.count())
print("Observations in testing set = ", testDF.count())

Observations in training set =  36680
Observations in testing set =  15495


### Supervised Learning - Classification 
#### Logistic Regression

In [35]:
# import the LogisticRegression function from the pyspark.ml.classification package
from pyspark.ml.classification import LogisticRegression

# Build the LogisticRegression object 'lr' by setting the required parameters
lr = LogisticRegression(featuresCol="features", labelCol="label",maxIter= 10,regParam=0.3, elasticNetParam=0.8)

# fit the LogisticRegression object on the training data
lrmodel = lr.fit(trainDF)

#This LogisticRegressionModel can be used as a transformer to perform prediction on the testing data
predictonDF = lrmodel.transform(testDF)

predictonDF.select("label","rawPrediction", "probability", "prediction").show(10,False)

+-----+--------------------------------------+-----------------------------------------+----------+
|label|rawPrediction                         |probability                              |prediction|
+-----+--------------------------------------+-----------------------------------------+----------+
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |


In [36]:
predictonDF.select("label","rawPrediction", "probability", "prediction").show(50,False)

+-----+--------------------------------------+-----------------------------------------+----------+
|label|rawPrediction                         |probability                              |prediction|
+-----+--------------------------------------+-----------------------------------------+----------+
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |
|0.0  |[4.026156743176436,-4.026156743176436]|[0.9824700109051254,0.017529989094874576]|0.0       |


### Model Evaluation

In [37]:
# import BinaryClassificationEvaluator from the pyspark.ml.evaluation package
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Build the BinaryClassificationEvaluator object 'evaluator'
evaluator = BinaryClassificationEvaluator()

# Calculate the accracy and print its value
accuracy = predictonDF.filter(predictonDF.label == predictonDF.prediction).count()/float(predictonDF.count())
print("Accuracy = ", accuracy)

# evaluate(predictiondataframe) gets area under the ROC curve
print('Area under the ROC curve = ', evaluator.evaluate(predictonDF))

Accuracy =  0.98134882220071
Area under the ROC curve =  0.5


In [38]:
# Create model summary object
lrmodelSummary = lrmodel.summary

# Print the following metrics one by one: 
# 1. Accuracy
# Accuracy is a model summary parameter
print("Accuracy = ", lrmodelSummary.accuracy)
# 2. Area under the ROC curve
# Area under the ROC curve is a model summary parameter
print("Area under the ROC curve = ", lrmodelSummary.areaUnderROC)
# 3. Precision (Positive Predictive Value)
# Precision is a model summary parameter
print("Precision = ", lrmodelSummary.weightedPrecision)
# 4. Recall (True Positive Rate)
# Recall is a model summary parameter
print("Recall = ", lrmodelSummary.weightedRecall)
# 5. F1 Score (F-measure)
# F1 Score is a model summary method
print("F1 Score = ", lrmodelSummary.weightedFMeasure())

Accuracy =  0.9824700109051254
Area under the ROC curve =  0.5
Precision =  0.9652473223279173
Recall =  0.9824700109051254
F1 Score =  0.973782520813235
